<a href="https://colab.research.google.com/github/efkomarova/Sirius/blob/main/Structural_Patterns.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Структурные паттерны (Structural)

часть 2: **как соединять объекты**.

- **Adapter** — переходник (старый класс под новый интерфейс)
- **Bridge** — абстракция и реализация раздельно
- **Composite** — дерево с одинаковым интерфейсом
- **Decorator** — обёртки добавляют поведение
- **Facade** — один вход в сложное
- **Flyweight** — общее в кэше
- **Proxy** — заместитель с отложенной загрузкой

## 6. Adapter — переходник

**Легенда:** Купил прибор в другой стране — вилка не подходит. Берёшь переходник. Он «переводит»: с одной стороны — твоя вилка, с другой — наша розетка.

**Зачем нужен:** Есть старый класс, но интерфейс не совпадает с новым кодом. Адаптер — прослойка: снаружи выдаёт нужный интерфейс, внутри вызывает старый класс.

**Ключ в коде:** Новый класс наследует нужный интерфейс, содержит старый класс, методы делегируют.

In [1]:
%%writefile adapter.cpp
/* ЛЕГЕНДА: Переходник — старый шнур не влезает в розетку. СУТЬ: «переводит» старый интерфейс под новый. */

#include <iostream>
using namespace std;

// Новый интерфейс — клиент ожидает метод plug()
class NewSocket {
public:
    virtual void plug(const string& device) = 0;
    virtual ~NewSocket() = default;
};

// Старый класс — другой метод (plugOld вместо plug). Переписывать его не хотим.
class OldDevice {
public:
    void plugOld(const string& d) { cout << "Подключено: " << d << endl; }
};

// АДАПТЕР: наследует NewSocket (выглядит как новый), внутри содержит OldDevice
// plug() «переводит» вызов в plugOld() — делегирует старому классу
class Adapter : public NewSocket {
    OldDevice old;  // композиция — храним старый прибор внутри
public:
    void plug(const string& d) override { old.plugOld(d); }  // адаптация!
};

int main() {
    NewSocket* s = new Adapter();  // клиент работает с NewSocket, не знает про OldDevice
    s->plug("телефон");            // внутри вызовется plugOld("телефон")
    delete s;
}

Writing adapter.cpp


In [2]:
!g++ -std=c++17 -o out adapter.cpp && ./out

Подключено: телефон


## 7. Bridge — две вещи раздельно

**Легенда:** Одна песня — слушаешь через колонки или наушники. Музыка одна, способ вывода разный. Меняются независимо.

**Зачем нужен:** Две «оси» изменений. Формы (круг, квадрат) и способы рисования (карандаш, кисть). С Bridge — 2+2 класса, комбинируются. Без — 4 класса под каждую комбинацию.

In [3]:
%%writefile bridge.cpp
/* ЛЕГЕНДА: Одна песня — через колонки или наушники. Музыка та же, вывод разный. СУТЬ: абстракция и реализация раздельно. */

#include <iostream>
using namespace std;

// РЕАЛИЗАЦИЯ — куда выводим звук. Меняется независимо от Player
class AudioOut {
public:
    virtual void play(const string& track) = 0;
    virtual ~AudioOut() = default;
};
class Speakers : public AudioOut {
    void play(const string& s) override { cout << "Колонки: " << s << endl; }
};
class Headphones : public AudioOut {
    void play(const string& s) override { cout << "Наушники: " << s << endl; }
};

// АБСТРАКЦИЯ — плеер хранит ссылку на способ вывода
// Player не знает, колонки это или наушники — просто делегирует out->play()
class Player {
    AudioOut* out;  // указатель на реализацию (внедрение зависимости)
public:
    Player(AudioOut* o) : out(o) {}  // &sp или &hp — передаём при создании
    void playTrack(const string& name) { out->play(name); }
};

int main() {
    Speakers sp;
    Headphones hp;
    Player p1(&sp), p2(&hp);  // & — адрес переменной (указатель)
    p1.playTrack("Песня.mp3");  // колонки
    p2.playTrack("Песня.mp3");  // наушники — один код, разное поведение
}

Writing bridge.cpp


In [4]:
!g++ -std=c++17 -o out bridge.cpp && ./out

Колонки: Песня.mp3
Наушники: Песня.mp3


## 8. Composite — дерево одинаковых

**Легенда:** Папка в компьютере. Внутри — файлы и папки. «Открываешь» и то, и другое. Действие одно, поведение разное. Папка показывает детей — рекурсия.

**Зачем нужен:** Древовидная структура (меню, форма, документ). Обрабатываем листья и ветки одинаково, без проверок «это файл или папка?»

In [5]:
%%writefile composite.cpp
/* ЛЕГЕНДА: Папка — внутри файлы и папки. Открываешь одинаково. СУТЬ: листья и группы — один интерфейс, древовидная структура. */

#include <iostream>
#include <vector>  // динамический массив
using namespace std;

// Базовый узел — и файл, и папка умеют show(). Единый интерфейс!
class FileNode {
public:
    virtual void show() = 0;
    virtual ~FileNode() = default;
};

// ЛИСТ — конечный элемент (файл), нет детей
class File : public FileNode {
    string name;
public:
    File(const string& n) : name(n) {}
    void show() override { cout << "  " << name << endl; }
};

// КОНТЕЙНЕР — папка содержит другие узлы (файлы или папки)
class Folder : public FileNode {
    string name;
    vector<FileNode*> items;  // массив указателей — и File, и Folder
public:
    Folder(const string& n) : name(n) {}
    void add(FileNode* n) { items.push_back(n); }  // push_back = добавить в конец
    void show() override {
        cout << "[" << name << "]" << endl;
        for (auto i : items)  // цикл по всем элементам
            i->show();       // рекурсия! если item — Folder, его show() пойдёт по детям
    }
};

int main() {
    Folder f("Документы");
    f.add(new File("doc.txt"));
    f.add(new File("readme.txt"));
    cout << "Папка:" << endl;
    f.show();
}

Writing composite.cpp


In [6]:
!g++ -std=c++17 -o out composite.cpp && ./out

Папка:
[Документы]
  doc.txt
  readme.txt


## 9. Decorator — обёртки

**Легенда:** Бутерброд. Хлеб (1 слой), масло (2), сыр (3). Каждая добавка — обёртка. Можно добавлять бесконечно.

**Зачем нужен:** Добавлять объекту новые возможности без подклассов под каждую комбинацию. Кофе: эспрессо +молоко +сироп — комбинируешь на лету.

**Ключ:** Декоратор хранит указатель на «внутренний» объект, свой метод = base->method() + своё.

In [7]:
%%writefile decorator.cpp
/* ЛЕГЕНДА: Бутерброд — хлеб, масло, сыр. Каждый слой добавляет. СУТЬ: обёртки добавляют поведение, комбинируются на лету. */

#include <iostream>
using namespace std;

// Базовый интерфейс — «бутерброд» умеет сообщить количество слоёв
class Sandwich {
public:
    virtual int layers() = 0;
    virtual ~Sandwich() = default;
};

// Основа — один слой (хлеб). Лист цепочки.
class Bread : public Sandwich {
    int layers() override { return 1; }
};

// Декоратор — обёртка. Хранит «внутренний» Sandwich, добавляет свой слой
class Topping : public Sandwich {
protected:
    Sandwich* base;  // на кого «намазываем»
public:
    Topping(Sandwich* b) : base(b) {}
};

// Cheese — декоратор. Свой layers() = base->layers() + 1
class Cheese : public Topping {
public:
    Cheese(Sandwich* b) : Topping(b) {}
    int layers() override { return base->layers() + 1; }  // делегируем + добавляем
};

int main() {
    // Bread + Cheese + Cheese = 3 слоя. Вложенные обёртки!
    Sandwich* s = new Cheese(new Cheese(new Bread()));
    cout << "Слоёв: " << s->layers() << endl;  // вызов идёт по цепочке: Cheese->Cheese->Bread
    delete s;
}

Writing decorator.cpp


In [8]:
!g++ -std=c++17 -o out decorator.cpp && ./out

Слоёв: 3


## 10. Facade — одна кнопка

**Легенда:** Микроволновка. Нажал «Разогреть» — внутри нагрев, вращение, таймер. Одна кнопка — всё работает.

**Зачем нужен:** Сложная система из многих классов. Facade даёт один простой метод, внутри вызывает нужные методы подсистемы.

In [9]:
%%writefile facade.cpp
/* ЛЕГЕНДА: Микроволновка — нажал «Разогреть». Внутри много действий. СУТЬ: простой интерфейс к сложной подсистеме. */

#include <iostream>
using namespace std;

// Сложная подсистема — много классов с разными методами
class Heater     { public: void heat()   { cout << "Нагрев\n"; } };
class Turntable  { public: void spin()  { cout << "Вращение\n"; } };
class Timer      { public: void run()   { cout << "Таймер\n"; } };

// ФАСАД — один класс объединяет подсистему. Один метод warmUp() вместо трёх
class Microwave {
    Heater h;
    Turntable t;
    Timer ti;
public:
    void warmUp() {
        h.heat();   // последовательно вызываем методы подсистемы
        t.spin();
        ti.run();
    }
};

int main() {
    Microwave mw;
    mw.warmUp();  // пользователь — один вызов вместо знания про Heater, Turntable, Timer
}

Writing facade.cpp


In [10]:
!g++ -std=c++17 -o out facade.cpp && ./out

Нагрев
Вращение
Таймер


## 11. Flyweight — образец один

**Легенда:** Буква «А» в документе 500 раз. Храним один образец. Для каждой позиции — ссылка на образец + координаты.

**Зачем нужен:** Экономия памяти. Общее — в одном месте. Уникальное — параметром.

In [11]:
%%writefile flyweight.cpp
/* ЛЕГЕНДА: Буква А — в шрифте одна, в тексте сотни раз. СУТЬ: общее кэшируется, уникальное (позиция) передаётся параметром. */

#include <iostream>
#include <map>  // словарь: ключ -> значение
using namespace std;

// Letter — образец буквы (общая часть). Один объект на символ.
class Letter {
    char ch;
public:
    Letter(char c) : ch(c) {}
    void draw(int pos) {  // pos — уникальное для каждого использования (внешнее состояние)
        cout << ch << " @ позиция " << pos << endl;
    }
};

// Кэш — храним один объект на каждую букву. find() ищет, end() = «не найден»
map<char, Letter*> letterCache;

Letter* getLetter(char c) {
    if (letterCache.find(c) == letterCache.end())
        letterCache[c] = new Letter(c);  // создаём только если ещё нет
    return letterCache[c];
}

int main() {
    getLetter('A')->draw(1);   // 'A' — один объект, позиция 1
    getLetter('A')->draw(5);   // тот же объект Letter('A'), другая позиция!
    getLetter('B')->draw(10);
}

Writing flyweight.cpp


In [12]:
!g++ -std=c++17 -o out flyweight.cpp && ./out

A @ позиция 1
A @ позиция 5
B @ позиция 10


## 12. Proxy — замена с откладыванием

**Легенда:** Галерея фото. Маленькие превью грузятся быстро. Кликнул — загружается полноразмерное. До клика тяжёлый файл не трогаем.

**Зачем нужен:** Создание объекта дорогое. Proxy — «заглушка»: создаётся быстро. При первом использовании создаёт настоящий и делегирует ему.

In [13]:
%%writefile proxy.cpp
/* ЛЕГЕНДА: Превью фото — маленькая картинка. Кликнул — загрузилась большая. СУТЬ: откладывает создание тяжёлого объекта до первого использования. */

#include <iostream>
using namespace std;

// Интерфейс — и превью, и полная картинка умеют get()
class Product {
public:
    virtual void get() = 0;
    virtual ~Product() = default;
};

// Реальный объект — ТЯЖЁЛЫЙ. Загрузка происходит в конструкторе!
class FullImage : public Product {
    string name;
public:
    FullImage(const string& n) : name(n) { cout << "Загрузка " << n << endl; }  // дорого!
    void get() override { cout << "Показано " << name << endl; }
};

// ПРОКСИ — лёгкий. Создаёт FullImage только при первом вызове get()
class Thumbnail : public Product {
    string name;
    FullImage* full = nullptr;  // изначально nullptr — не загружаем
public:
    Thumbnail(const string& n) : name(n) { cout << "Превью: " << n << endl; }  // быстро!
    void get() override {
        if (!full)
            full = new FullImage(name);  // ленивая загрузка — только при первом клике
        full->get();  // делегируем реальному объекту
    }
    ~Thumbnail() { delete full; }  // деструктор — освобождаем память
};

int main() {
    Product* p = new Thumbnail("photo.jpg");  // здесь только превью, загрузки нет
    p->get();  // вот здесь произойдёт загрузка FullImage!
    delete p;
}

Writing proxy.cpp


In [14]:
!g++ -std=c++17 -o out proxy.cpp && ./out

Превью: photo.jpg
Загрузка photo.jpg
Показано photo.jpg
